# THE BATTLE OF NEIGHBOURHOOD
## IBM Data science 
 **Creator: Dheemanth shenoy**
***

## Business problem

The objective of this capstone project is to analyse and select the best locations in the city of Kuala Lumpur, Malaysia to open a new shopping mall. Using data science methodology and machine learning techniques like clustering, this project aims to provide solutions to answer the business.
 <br>In the city of Kuala Lumpur, Malaysia, if a property developer is looking to open a new shopping mall, where would you recommend that they open it?
 ***

## Data

1. List of neighbourhoods in Kuala Lumpur. This defines the scope of this project which is confined to the city of Kuala Lumpur, the capital city of the country of Malaysia in South East Asia.
1. Latitude and longitude coordinates of those neighbourhoods. This is required in order to plot the map and also to get the venue data.
1. Venue data, particularly data related to shopping malls. We will use this data to perform clustering on the neighbourhoods.
***

In [67]:
import pandas as pd 
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
!pip install geocoder
from geopy.geocoders import Nominatim 
import geocoder
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium 
print("Libraries imported.")

Libraries imported.


In [7]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [8]:
kl_df.shape

(71, 1)

In [11]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
        return lat_lng_coords

In [12]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]
coords

[[3.0576900000000364, 101.74388000000005],
 [3.148494115588384, 101.69672876508707],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1511418740671178, 101.69937468268016],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.136442497863743, 101.69029641203808],
 [3.1419057131529953, 101.67967833034047],
 [3.1566851613075664, 101.6

In [13]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
print(kl_df.shape)
kl_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


In [14]:
kl_df.to_csv("kl_df.csv", index=False)

In [15]:
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [69]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [17]:
map_kl.save('map_kl.html')

In [72]:
CLIENT_ID = 'Your ID' 
CLIENT_SECRET = 'Your SECRET' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Your ID
CLIENT_SECRET:Your SECRET


In [36]:
radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [34]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(7084, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant


In [35]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [47]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
venues_df['VenueCategory'].unique()[:304]

There are 304 uniques categories.


array(['Supplement Shop', 'Food & Drink Shop', 'Noodle House',
       'Seafood Restaurant', 'Chinese Restaurant', 'Breakfast Spot',
       'Restaurant', 'Food Court', 'Dim Sum Restaurant',
       'Other Great Outdoors', 'Vegetarian / Vegan Restaurant', 'Park',
       'Indian Restaurant', 'Coffee Shop', 'Asian Restaurant',
       'Bubble Tea Shop', 'Spa', 'Convenience Store', 'Food Truck',
       'Snack Place', 'Chinese Breakfast Place', 'Outlet Store',
       'Dessert Shop', 'Pet Store', 'Cantonese Restaurant', 'Café',
       'Bakery', 'Japanese Restaurant', 'Malay Restaurant',
       'Gym / Fitness Center', 'Farmers Market', 'Steakhouse',
       'Hakka Restaurant', 'Fast Food Restaurant', 'Athletics & Sports',
       'Badminton Court', 'Mamak Restaurant', 'Burger Joint', 'Winery',
       'College Bookstore', 'Grocery Store', 'Shop & Service',
       'Halal Restaurant', 'Hostel', 'Exhibit', 'South Indian Restaurant',
       'Chettinad Restaurant', 'Hotel', 'Juice Bar', 'Speakeasy',
   

In [50]:
"Neighborhood" in venues_df['VenueCategory'].unique()

False

In [51]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7084, 305)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tour

In [52]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(71, 305)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tour

In [53]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

38

In [57]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.00
2,Bandar Menjalara,0.01
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.01


In [58]:
kclusters = 3
kl_clustering = kl_mall.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 2, 2, 1, 0], dtype=int32)

In [59]:
kl_merged = kl_mall.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,0
1,"Ampang, Kuala Lumpur",0.00,0
2,Bandar Menjalara,0.01,0
3,Bandar Sri Permaisuri,0.00,0
4,Bandar Tasik Selatan,0.01,0


In [60]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() 

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.00,0,3.148494,101.696729
2,Bandar Menjalara,0.01,0,3.190350,101.625450
3,Bandar Sri Permaisuri,0.00,0,3.103910,101.712260
4,Bandar Tasik Selatan,0.01,0,3.072750,101.714610


In [61]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0,3.057690,101.743880
33,Kampung Padang Balang,0.00,0,3.209430,101.693180
34,Kepong,0.00,0,3.217500,101.637630
69,Titiwangsa,0.01,0,3.180730,101.703210
37,Maluri,0.00,0,3.147890,101.694050
39,Miharja,0.00,0,3.147890,101.694050
41,Pantai Dalam,0.00,0,3.094760,101.667470
44,Salak South,0.00,0,3.081020,101.697240
46,Semarak,0.00,0,3.179916,101.721437
47,"Sentul, KL",0.01,0,3.175080,101.693050


In [70]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
map_clusters.save('map_clusters.html')

In [64]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0,3.057690,101.743880
33,Kampung Padang Balang,0.00,0,3.209430,101.693180
34,Kepong,0.00,0,3.217500,101.637630
69,Titiwangsa,0.01,0,3.180730,101.703210
37,Maluri,0.00,0,3.147890,101.694050
39,Miharja,0.00,0,3.147890,101.694050
41,Pantai Dalam,0.00,0,3.094760,101.667470
44,Salak South,0.00,0,3.081020,101.697240
46,Semarak,0.00,0,3.179916,101.721437
47,"Sentul, KL",0.01,0,3.175080,101.693050


In [65]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Bangsar South,0.02,1,3.111020,101.662830
11,Brickfields,0.03,1,3.129160,101.684060
12,Bukit Bintang,0.02,1,3.147770,101.708550
66,Taman Tun Dr Ismail,0.03,1,3.152830,101.622710
30,KL Eco City,0.03,1,3.117130,101.673840
51,Sri Hartamas,0.02,1,3.162200,101.650360
31,"Kampung Baru, Kuala Lumpur",0.03,1,3.165460,101.710280
27,"Jalan Cochrane, Kuala Lumpur",0.02,1,3.132903,101.724679
38,Medan Tuanku,0.02,1,3.159260,101.698340
22,"Damansara, Kuala Lumpur",0.03,1,3.141906,101.679678


In [66]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
42,"Pudu, Kuala Lumpur",0.04,2,3.133540,101.713070
36,Lembah Pantai,0.05,2,3.121202,101.663899
67,Taman U-Thant,0.04,2,3.157700,101.724520
7,Bangsar Park,0.05,2,3.129200,101.678440
6,Bangsar,0.05,2,3.129200,101.678440


## Observation
Most of the Shopping malls are concentrated in the central area of kuala, lumpur city. The moderate is at cluster 1 and Highest is at cluster 2.The lowest is at Cluster 0 where the competition is less, so starting a new shopping mall will be profitable in cluster 0. Cluster 1 and 2 will be less profitable. Beacuse there is already competetions going on, bringing on a new shopping mall will not a practical thing. 